Download alignment from pfam for ANF_receptor (PF01094) from http://pfam.xfam.org/family/PF01094#tabview=tab3 V34.0.

It is saved as the PF01094_full.an following Ranganathan lab formatting. There are some differences though between the files. I believe the alignment files for their notebooks are already annotated when they are downlaoded in the runAllNBCalcs.sh bash script. (there is a not \_full.an file that has the normal headers from pfam) I need to do this for mine. 

For every 'chunk' that annotates a protein family there is some line that begins with 'tee' and specifie sthe log file. This enables the outputting of the bash prompt to the file.

There is also a getpfamdb.sh file that will download the pfam taxonomy database.

But let's skip all the fancy nonsense of pfam annotations and just dive right in. 

In [1]:
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import copy
import scipy.cluster.hierarchy as sch
from scipy.stats import scoreatpercentile 
import matplotlib.image as mpimg
from IPython.display import display
from IPython.display import Image
from Bio.Seq import Seq
from Bio import motifs
import colorsys
from pysca import scaTools as sca
# import mpld3
import pickle as pickle
from optparse import OptionParser

%matplotlib inline

if not os.path.exists('../output/'):
    os.makedirs('../output/')  

To begin, we read in the results of the above three scripts (scaProcessMSA, scaCore and scaSectorID), stored as three dictionaries in the database PF00071_full.db.  To see what variables are stored in each dictionary, use the command dictionary.keys( ), e.g.:

    >>> print Dseq.keys()

In [3]:
db = pickle.load(open('../output/PF00071_rd2.db','rb'))
Dseq = db['sequence']  #the results of scaProcessMSA
Dsca = db['sca']       #the results of scaCore
Dsect = db['sector']   #the results of scaSectorID